# Zoning code searching
- This document handles zoning code searching in the zoning ordinates (finding out if a "code" appeared in a document)
- Part 1: codelibrary 
    - 1.1 scraping: fake api calls
    - 1.2 load existing zoning names/codes and find them in the documents
- Part 2: municode

In [1]:
import numpy as np
import pandas as pd
import re
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import pickle
import string
printable_set = set(list(string.printable))

# 1. codelibrary

In [2]:
user_agent = 'Mozilla/5.0 (Macintosh; U; Intel Mac OS X 10_6_6; en-US) AppleWebKit/533.20.25 (KHTML, like Gecko) Version/5.0.4 Safari/533.20.27'
# url = 'https://codelibrary.amlegal.com/codes/templecityca/latest/templecity_ca/0-0-0-34203'

headers = {
    'User-Agent': user_agent
}

In [3]:
# temple city
# LOCKPORT, il
# use zoning list instead of the whole zoning (for parsing)

## 1.1 Scraping

In [4]:
idx = 190
url = 'https://codelibrary.amlegal.com/api/render-section/winthropharbor/latest/winthropharbor_il/0-0-0-39885/{}/'.format(idx)   
url
res = requests.get(url, headers = headers)
soup = BeautifulSoup(res.text, features="lxml")
res.status_code == 404

False

In [5]:
def clean_content(sent):
    sent = ''.join([c if c in printable_set else ' ' for c in sent])
    sent = re.sub('\n', ' ', sent)
    sent = re.sub(' +', ' ', sent)
    sent = re.sub('\t+', ' ', sent)
    return sent

def change_url(url):
    if '#' in url:
        url = url.split('#')[0]
    url = re.sub('codes', 'api/render-section', url)
    url = url
    return url

def process_soup_text(soup_text):
    starting_idx = list(re.finditer('"html":"', soup_text))[0].span()[1]
    return soup_text[starting_idx:]

def parse_table(soup):
    table_text = []
    for t in soup.find_all('table'):
        for child in t.tbody.children:
            table_text.append(' '.join([i.text for i in list(child.children)]))
    if len(table_text) == 0:
        return ''
    return '\n'.join(table_text)

def parse_codelibrary(url):
    res = requests.get(url, headers = headers)
    soup = BeautifulSoup(res.text, features="lxml")

    i = 0
    start_scraping = False
    section_count = 0
    all_text = []
    
    curr_section = soup.find_all('div', attrs = {'id': 'curr-section'})
    if len(curr_section) == 0:
        print('[ERROR!] No current_section found! URL:', url)
        return ''
    
    curr_section_found = False
    starting_id = 0
    for child in curr_section[0].children:
        try:
            starting_id = child['id'].replace('section-', '').strip()
            starting_id = int(starting_id)
            curr_section_found = True
        except:
            continue
        if curr_section_found:
            all_text.append(child.text)
            section_count += 1
            break
    
    i = starting_id + 1
    while True:
        if not start_scraping:
            section_i = soup.find_all('div', attrs = {'id': 'section-{}'.format(i)})
            #print(i, len(section_i))
            if len(section_i) == 0:
                start_scraping = True
            else:
                all_text.append(section_i[0].text)
                section_count += 1
                i += 1
        else:
            section_url = change_url(url) + '/{}/'.format(i)
            try:
                res = requests.get(section_url, headers = headers)
            except:
                break
            #print(section_url)
            #print(i, res.status_code)
            if res.status_code == 404:
                break
            soup = BeautifulSoup(res.text, features="lxml")
            table_text = parse_table(soup)
            tmp_text_result = process_soup_text(soup.text) + '\n' + table_text
            all_text.append(tmp_text_result)
            section_count += 1
            if section_count > 10:
                break
            i += 1
    return '\n'.join([clean_content(doc) for doc in all_text]), section_count
    #return all_text, section_count

In [4]:
# def clean_content(sent):
#     sent = ''.join([c if c in printable_set else ' ' for c in sent])
#     sent = re.sub('\n', ' ', sent)
#     sent = re.sub(' +', ' ', sent)
#     sent = re.sub('\t+', ' ', sent)
#     return sent

# def change_url(url):
#     if '#' in url:
#         url = url.split('#')[0]
#     url = re.sub('codes', 'api/render-section', url)
#     url = url
#     return url

# def process_soup_text(soup_text):
#     #print(soup_text)
#     #print('"html":"' in soup_text)
#     #print(list(re.finditer('"html":"', soup_text)))
#     starting_idx = list(re.finditer('"html":"', soup_text))[0].span()[1]
#     return soup_text[starting_idx:]

# def parse_codelibrary(url):
#     res = requests.get(url, headers = headers)
#     soup = BeautifulSoup(res.text, features="lxml")

#     i = 0
#     start_scraping = False
#     section_count = 0
#     all_text = []
    
#     curr_section = soup.find_all('div', attrs = {'id': 'curr-section'})
#     if len(curr_section) == 0:
#         print('[ERROR!] No current_section found! URL:', url)
#         return ''
    
#     curr_section_found = False
#     starting_id = 0
#     for child in curr_section[0].children:
#         try:
#             starting_id = child['id'].replace('section-', '').strip()
#             starting_id = int(starting_id)
#             curr_section_found = True
#         except:
#             continue
#         if curr_section_found:
#             all_text.append(child.text)
#             section_count += 1
#             break
    
#     i = starting_id + 1
#     while True:
#         if not start_scraping:
#             section_i = soup.find_all('div', attrs = {'id': 'section-{}'.format(i)})
#             #print(i, len(section_i))
#             if len(section_i) == 0:
#                 start_scraping = True
#             else:
#                 all_text.append(section_i[0].text)
#                 section_count += 1
#                 i += 1
#         else:
#             section_url = change_url(url) + '/{}/'.format(i)
#             try:
#                 res = requests.get(section_url, headers = headers)
#             except:
#                 break
#             #print(section_url)
#             #print(i, res.status_code)
#             if res.status_code == 404:
#                 break
#             soup = BeautifulSoup(res.text, features="lxml")
#             all_text.append(process_soup_text(soup.text))
#             section_count += 1
#             i += 1
#     return '\n'.join([clean_content(doc) for doc in all_text]), section_count

- **testing scraper**

In [6]:
# url = 'https://codelibrary.amlegal.com/codes/lockport/latest/lockport_il/0-0-0-77949'
# lockport, section_count = parse_codelibrary(url)

# url = 'https://codelibrary.amlegal.com/codes/danvilleca/latest/danville_ca/0-0-0-7018'
# danville_ca, section_count = parse_codelibrary(url)

url = 'https://codelibrary.amlegal.com/codes/minookail/latest/minooka_il/0-0-0-5147'
minoka, section_count = parse_codelibrary(url)

In [7]:
section_count

4

In [8]:
minoka

'CHAPTER 4ZONING DISTRICTS AND MAPSECTION:5-4-1 Establishment Of Districts5-4-2 Zoning District Map; Boundaries Of Districts5-4-3 Annexed Territory \n5-4-1: ESTABLISHMENT OF DISTRICTS:In order to carry out the purposes and intent of this title, the village is hereby divided into the following districts:AAgricultural districtR1Single-family detached residence districtR1ASingle-family detached residence districtR2Single-family detached residence districtR3Single-family attached and multiple-family residence districtR4Two-family (duplex) residence districtR4ATwo-family (duplex) residence districtR5Single-family attached residence districtR6Multiple-family residence districtB1Business districtB2Commercial districtM1Manufacturing districtM2Manufacturing districtLowland conservancy overlay district (Ord. 2003-34, 8-26-2003)\\n"} A Agricultural district R1 Single-family detached residence district R1A Single-family detached residence district R2 Single-family detached residence district R3 Si

In [363]:
# https://codelibrary.amlegal.com/api/render-section/lockport/latest/lockport_il/0-0-0-77949/2/

In [364]:
# res = requests.get(url, headers = headers)
# soup = BeautifulSoup(res.text, features="lxml")

In [365]:
# res.status_code

## 1.2 Matching existing oridinates

In [10]:
za_ordinates_links = pd.read_csv('../data/za_ordinates_links.csv')
za_ordinates_links.head(3)

,assignedto,ID,ID_notunique,GEOID,state,county,place,muncipalcode,site,priority1129,...,Use for model,complex?,zoningchapter,List of Zoning Districts,Residential_density_section,Table or text,Zoning chapter or density table as PDF? (new field),Other district density section,Notes,Unnamed: 20
0,Jacob,NaN,30,151696,Alabama,Jefferson County,MOUNTAIN BROOK,https://library.municode.com/al/mountain_brook,Municode,0.0,...,1.0,NaN,https://library.municode.com/al/mountain_brook...,https://library.municode.com/al/mountain_brook...,NaN,text,NaN,NaN,different density section for each district,NaN
1,Jacob,NaN,16,100820,Alabama,Shelby County,ALABASTER,https://library.municode.com/al/alabaster,Municode,0.0,...,1.0,NaN,https://library.municode.com/al/alabaster/code...,https://library.municode.com/al/alabaster/code...,https://library.municode.com/al/alabaster/code...,multiple tables,NaN,NaN,NaN,NaN
2,Jacob,NaN,135,444410,Arizona,Pinal County,MARICOPA,https://maricopa.municipal.codes/,Codebook,1.0,...,1.0,NaN,https://maricopa.municipal.codes/MCC/18,https://maricopa.municipal.codes/MCC/18.10.010,https://maricopa.municipal.codes/MCC/18.35.030,table,NaN,NaN,"different density table for rural, residential...",NaN


In [11]:
za_ordinates_links.isna().sum()
# len(all_results), i

assignedto                                               1
ID                                                     193
ID_notunique                                             1
GEOID                                                    1
state                                                    1
county                                                   1
place                                                    1
muncipalcode                                             1
site                                                     1
priority1129                                             1
priority1130                                             1
Use for model                                           22
complex?                                               110
zoningchapter                                           34
List of Zoning Districts                                51
Residential_density_section                             53
Table or text                                           

In [12]:
%run ../src/tree_spider.py
%run ../src/utils.py

In [60]:
all_results = []
counter = 0
# starting = 112
weird_county = []

In [61]:
for i in tqdm(range(za_ordinates_links.shape[0])):
#     if i < starting:
#         continue
    curr_row = za_ordinates_links.iloc[i]
    url = curr_row['List of Zoning Districts']
    if isinstance(url, str) and 'codelibrary' in url:
        all_text, section_count = parse_codelibrary(url)
#         if section_count == 1:
#             url = curr_row['List of Zoning Districts']
#             #print(curr_row)
#             try:
#                 all_text, section_count = parse_codelibrary(url)
#             except:
#                 weird_county.append(i)
                
        all_results.append({
            'idx': counter,
            'state': curr_row.state,
            'county': curr_row.county,
            'muni': curr_row.place,
            'all_text': all_text,
            'section_count': section_count,
            'source': 'codelibrary'
        })
    elif isinstance(url, str) and 'ecode360' in url:
        try:
            ts = TreeSpider(url.split('/')[-1])
            ts.run()
            all_text = ts.find_all_leaf_text()
        except:
            all_text = 'ecode_360_nothing_found'
        section_count = -1
        all_results.append({
            'idx': counter,
            'state': curr_row.state,
            'county': curr_row.county,
            'muni': curr_row.place,
            'all_text': all_text,
            'section_count': section_count,
            'source': 'ecode360'
        })
        
    counter += 1

100%|██████████| 193/193 [00:36<00:00,  5.26it/s]


In [62]:
# for i in tqdm(range(za_ordinates_links.shape[0])):
# #     if i < starting:
# #         continue
#     curr_row = za_ordinates_links.iloc[i]
#     url = curr_row['zoningchapter']
#     if isinstance(url, str) and 'codelibrary' in url:
#         all_text, section_count = parse_codelibrary(url)
#         if section_count == 1:
#             url = curr_row['List of Zoning Districts']
#             #print(curr_row)
#             try:
#                 all_text, section_count = parse_codelibrary(url)
#             except:
#                 weird_county.append(i)
                
#         all_results.append({
#             'idx': counter,
#             'state': curr_row.state,
#             'county': curr_row.county,
#             'muni': curr_row.place,
#             'all_text': all_text,
#             'section_count': section_count,
#             'source': 'codelibrary'
#         })
#     elif isinstance(url, str) and 'ecode360' in url:
#         try:
#             ts = TreeSpider(url.split('/')[-1])
#             ts.run()
#             all_text = ts.find_all_leaf_text()
#         except:
#             all_text = 'ecode_360_nothing_found'
#         section_count = -1
#         all_results.append({
#             'idx': counter,
#             'state': curr_row.state,
#             'county': curr_row.county,
#             'muni': curr_row.place,
#             'all_text': all_text,
#             'section_count': section_count,
#             'source': 'ecode360'
#         })
        
#     counter += 1

In [63]:
# ts.root_node.children[0]

In [64]:
codelibrary_results = pd.DataFrame(all_results)
codelibrary_results.head(7)

,idx,state,county,muni,all_text,section_count,source
0,3,California,Contra Costa County,DANVILLE,ShareDownloadBookmarkPrintARTICLE VZONING MAP;...,1,codelibrary
1,6,California,Los Angeles County,TEMPLE CITY,ShareDownloadBookmarkPrint9-1B-1: ESTABLISHMEN...,1,codelibrary
2,22,Florida,Broward County,COOPER CITY,Chapter 23ZONING DISTRICTS**Editor's note Unif...,11,codelibrary
3,31,Idaho,Canyon County,MIDDLETON,ShareDownloadBookmarkPrint5-1-4: ZONING DISTRI...,1,codelibrary
4,33,Illinois,Cook County,WESTERN SPRINGS,CHAPTER 5ZONING DISTRICTSSECTION:10-5-1: Estab...,5,codelibrary
5,34,Illinois,DuPage County,BARTLETT,ShareDownloadBookmarkPrint10-4-1: DISTRICTS ES...,1,codelibrary
6,35,Illinois,DuPage County,HINSDALE,ShareDownloadBookmarkPrintARTICLE II. ZONING D...,1,codelibrary


In [65]:
codelibrary_results.query('muni == "MINOOKA"').all_text

7    CHAPTER 4ZONING DISTRICTS AND MAPSECTION:5-4-1...
Name: all_text, dtype: object

In [66]:
codelibrary_results.to_csv('../data/codelibrary_results_1208.csv', index = False)

In [67]:
zoning_code_julia = pd.read_csv('../data/zoning_codes_julia.csv')
zoning_code_julia.head(2)
# count_zoning >2

,ID,state,muni,county,zoning,zoning_name,likelyres,count_zoning
0,115.0,MD,Baltimore City,Baltimore,R-2,Detached and Semi-Detached Residential Zoning ...,1.0,99.0
1,168.0,MD,Oxford,Talbot,R-2,Historic Residential District,1.0,99.0


In [68]:
all_codes = zoning_code_julia.zoning.dropna().values
len(all_codes)

3213

In [69]:
# all_codes = [code for code in all_codes if len(code) > 1] #no need to ignore R, C? 
all_codes

array(['R-2', 'R-2', 'R-2', ...,
       'MPD(institutionalized care facilities for seniors: senior group homes)',
       'MPD(senior townhouse, patio houses, single-family detached, garden apts)',
       'MPD(multipurpose development)'], dtype=object)

In [70]:
# top_n = 200

unique_codes, counts = np.unique(all_codes, return_counts = True) #counts >2
all_codes_w_counts = dict(zip(unique_codes, counts))
# all_codes_w_counts = dict(sorted(all_codes_w_counts.items(), key = lambda x: x[1], reverse = True))

# top_n_codes = set(list(all_codes_w_counts.keys()))

In [71]:
all_codes_w_counts

{' RA (Two-family dwelling)': 1,
 '(Corner)': 4,
 '1': 1,
 '1-AC': 1,
 '1/2-AC': 1,
 '1/4-AC': 1,
 '2': 1,
 '3': 1,
 '3-AC': 1,
 '32': 1,
 '32W': 1,
 '34': 1,
 '35': 1,
 '36': 1,
 '38': 1,
 '4': 1,
 '5': 1,
 '5-AC': 1,
 '6': 1,
 '7': 1,
 '8': 1,
 '9': 1,
 'A': 11,
 'A (R )': 1,
 'A-1': 4,
 'A-1\xa0': 1,
 'A-2': 2,
 'A-5': 1,
 'A-6': 1,
 'A-C': 1,
 'A-G': 1,
 'A-O-B': 1,
 'A-R': 2,
 'A.S.': 1,
 'A/LC': 1,
 'AA-1': 1,
 'AA-2': 1,
 'AAH': 1,
 'AAR': 1,
 'AARC': 1,
 'AB': 1,
 'ABC': 1,
 'AC': 3,
 'AC-RCA': 1,
 'AET': 1,
 'AET ': 1,
 'AF': 1,
 'AF-1': 1,
 'AF-2': 1,
 'AG': 2,
 'AH': 3,
 'AH and RCA': 1,
 'AH-1': 7,
 'AH-2': 6,
 'AH-3': 7,
 'AH-3A': 1,
 'AH-3B': 1,
 'AH-4': 3,
 'AH-5': 2,
 'AH-6': 2,
 'AH-7': 2,
 'AH-8': 1,
 'AH-C': 1,
 'AHA': 1,
 'AHD-1': 1,
 'AHD-2': 1,
 'AHD-3A': 1,
 'AHD-3B': 1,
 'AHD-4': 1,
 'AHD-5': 1,
 'AHD-6': 1,
 'AHD-7': 1,
 'AHD-MU': 1,
 'AHO-A': 1,
 'AHOZ (Multi-family dwellings)': 1,
 'AHOZ (Townhouses)': 1,
 'AHOZ-2 (Apartments)': 1,
 'AHOZ-2 (Townhouses)': 1,


In [72]:
all_codes_highfreq = []
for(key,value) in all_codes_w_counts.items():
    if value>=1 :
        all_codes_highfreq.append(key)

In [73]:
all_codes_highfreq
len(all_codes_highfreq)


1910

In [74]:
# all_codes_w_counts
from nltk.tokenize import word_tokenize

In [75]:
all_counts_results = []
for row_idx in tqdm(range(codelibrary_results.shape[0])):
    curr_row = codelibrary_results.iloc[row_idx]
    all_text = curr_row.all_text
    all_tokens = np.array(word_tokenize(all_text))
    unique_counts = 0
    all_counts = 0
    matches = []
    for k in all_codes_highfreq:
        if k in all_tokens:
            unique_counts += 1
            all_counts += sum(all_tokens == k)
            matches.append(k)
    all_counts_results.append({
        'idx': curr_row.idx,
        'state': curr_row.state,
        'county': curr_row.county,
        'muni': curr_row.muni,
        'unique_matches': matches,
        'unique_counts': unique_counts,
        'all_counts': all_counts
    })

100%|██████████| 54/54 [00:02<00:00, 26.27it/s]


In [76]:
# type(word_tokenize(all_text))

In [77]:
all_counts_results_df = pd.DataFrame(all_counts_results)


In [78]:
all_counts_results_df

,idx,state,county,muni,unique_matches,unique_counts,all_counts
0,3,California,Contra Costa County,DANVILLE,[],0,0
1,6,California,Los Angeles County,TEMPLE CITY,"[A, C, I, IL, MU, OS, P, RH, RL, RM]",10,14
2,22,Florida,Broward County,COOPER CITY,"[1, 2, A, A-1, B-1, B-2, B-3, C-1, C-2, CS, E-...",25,38
3,31,Idaho,Canyon County,MIDDLETON,"[1, 2, A, B]",4,4
4,33,Illinois,Cook County,WESTERN SPRINGS,"[1, 2, 3, 4, 5, 6, 7, 8, A, B, C1, C2, DT, MXD...",21,41
5,34,Illinois,DuPage County,BARTLETT,"[A, B]",2,2
6,35,Illinois,DuPage County,HINSDALE,[],0,0
7,36,Illinois,Grundy County,MINOOKA,"[1, 2, 3, 4, A, B, B1, B2, R1, R1A, R2, R3, R4...",15,18
8,39,Illinois,Lake County,LINDENHURST,[],0,0
9,41,Illinois,Will County,MANHATTAN,"[1, 2, 3, 4, 5, 6, 7, 8, 9, A, A-1, B, BP, C-1...",31,888


In [79]:
# all_counts_results_df.muni.isna().sum()
za_ordinates_links['idx'] = range(za_ordinates_links.shape[0])

In [80]:
all_counts_results_df = all_counts_results_df.merge(za_ordinates_links[['idx', 'muncipalcode']], how = 'left', left_on = 'idx', right_on = 'idx')
def find_source(url):
    if 'codelibrary' in url:
        return 'codelibrary'
    if 'ecode360' in url:
        return 'ecode360'
    return 'NOT_FOUND'

all_counts_results_df['source'] = all_counts_results_df.muncipalcode.apply(find_source)

In [81]:
all_counts_results_df.query('source == "NOT_FOUND"')

,idx,state,county,muni,unique_matches,unique_counts,all_counts,muncipalcode,source


In [82]:
all_counts_results_df.to_csv('../data/all_counts_results_df_freqall_onlyzoninglist_1208.csv', index = False)

# 2. municode - not finished
- Hard to parse

In [160]:
user_agent = 'Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/27.0.1453.94 Safari/537.36'
headers = {
    'User-Agent': user_agent
}

In [161]:
# headers

In [260]:
url = 'https://library.municode.com/il/sauk_village/codes/code_of_ordinances?nodeId=MUCO_CH82ZOPLDE_ARTVIIZODI_DIV2REDI_S82-222BUYARE'
# url = 'https://api.municode.com/codesToc/breadcrumb?jobId=381269&nodeId=MUCO_CH82ZOPLDE_ARTVIIZODI_DIV2REDI_S82-222BUYARE&productId=15156'
# url = 'https://api.municode.com/codesToc?jobId=381269&nodeId=MUCO_CH82ZOPLDE_ARTIVZOAPAP&productId=15156'
# url = 'https://api.municode.com/CodesContent?jobId=381269&nodeId=MUCO_CH82ZOPLDE_ARTVIIZODI_DIV2REDI_S82-222BUYARE&productId=15156'
res = requests.get(url, headers = headers)
soup = BeautifulSoup(res.text, features="lxml")

In [261]:
res

<Response [200]>

In [179]:
# clean_content(soup.text)

In [1]:
# soup